In [769]:
%run data_plot.ipynb

In [770]:
aco_data = {'best_path': {},
            'possible_paths': {}}
solutions = [{}]

In [771]:
trail_data = {}
ant_data = {}

In [772]:
def init_ants(origin, coords_data, ants_count, ant_data):
    ants_count = math.ceil(ants_count / 2)
    dest_nodes = [x_y_base_station_1[0], x_y_base_station_2[0]]
    init_nodes = choice(np.array([i for i in coords_data.keys() if i not in dest_nodes]), ants_count, replace=False)
    # one of the origin nodes has to be the defined origin
    if origin not in init_nodes:
        init_nodes[-1] = origin
    i = 1
    for node in init_nodes:
        for dest_node in dest_nodes:
            ant_data[i] = {'start_node': node, 'destination_node': dest_node, 'paths': []}
            i += 1

    return ant_data

In [773]:
#init_ants(15, x_y_data, 20, ant_data)

In [774]:
def init_pheromone(coords_data, trail_data):
    for data_row in coords_data.keys():
        trail_row = []
        for data_col in coords_data.keys():
            if data_row == data_col:
                trail_row.append(0)
            else:
                if data_col > data_row:
                    trail_row.append(random.uniform(0, 1))
                else:
                    trail_row.append(trail_data[data_col][data_row - 1])
        trail_data[data_row] = trail_row

    return trail_data

In [775]:
#init_pheromone(x_y_data, trail_data)
#print(trail_data)

In [776]:
def calculate_route_costs(coords_data):
    bandwidth_data = []

    for data_row in coords_data.keys():
        # var for storing all path bandwidths & latencies
        path_costs = []
        for data_col in coords_data.keys():
            if data_row != data_col:
                # get path distance
                path_distance = calculate_distance(coords_data[data_row], coords_data[data_col])
                #print(f"Node {data_row} to Node {data_col}: {path_distance}")
                # get bandwidth between node and its next node and append it to path bandwidths var
                path_bandwidth = convert_distance_bandwidth(path_distance)
                #if path_bandwidth == 0:
                    #absolute_path_cost = 0
                #else:
                    #absolute_path_cost = round(absolute_cost_function(path_bandwidth, node_to_node_latency), 3)
                path_costs.append(path_bandwidth)
            else:
                path_costs.append(0)
        bandwidth_data.append(path_costs)

    return bandwidth_data

In [777]:
#bandwidth_data = calculate_route_costs(x_y_data)
#print(bandwidth_data)

In [778]:
def traverse_nodes(ant_data, coords_data, bandwidth_data, trail_data, alpha = 0.6):
    for ant in ant_data.keys():
        current_node = ant_data[ant]['start_node']
        ant_data[ant]['paths'].append({'path': [current_node], 'end-to-end_transmission': 0, 'latency': 0, 'cost': 0})
        # if the current path is not destination node
        while current_node != ant_data[ant]['destination_node']:
            # loop through all nodes and calculate sum of probabilities
            sum_of_probabilities = 0
            path_latency = 0
            for node in coords_data.keys():
                if node != current_node:
                    '''
                    calculate end to end bandwidth. First get all visited 
                    node in the path from origin up until the current node
                    '''
                    current_path = ant_data[ant]['paths'][-1]['path']
                    path_bandwidths = []
                    for path in range(len(current_path) - 1):
                        path_bandwidths.append(bandwidth_data[current_path[path] - 1][current_path[path + 1] - 1])
                    bandwidth = bandwidth_data[current_node - 1][node - 1]
                    path_bandwidths.append(bandwidth)
                    # compute end-to-end bandwidth rate
                    end_to_end_bandwidth = min(path_bandwidths)
                    path_latency += node_to_node_latency
                    # for each node further from original node, increase latency
                    if end_to_end_bandwidth == 0:
                        absolute_path_cost = 0
                    else:
                        # calculate absolute cost
                        absolute_path_cost = round(absolute_cost_function(bandwidth_data[current_node - 1][node - 1], path_latency), 3)
                    sum_of_probabilities += (trail_data[current_node][node - 1] * alpha) + \
                    ((1 - alpha) * absolute_path_cost)
            path_probabilities = [(None, 0)]
            # loop through all nodes and calculate probabilities
            for node in coords_data.keys():
                # dont calculate probability of visiting itself
                if node != current_node:
                    '''
                    calculate end to end bandwidth. First get all visited 
                    node in the path from origin up until the current node
                    '''
                    current_path = ant_data[ant]['paths'][-1]['path']
                    path_bandwidths = []
                    for path in range(len(current_path) - 1):
                        path_bandwidths.append(bandwidth_data[current_path[path] - 1][current_path[path + 1] - 1])
                    bandwidth = bandwidth_data[current_node - 1][node - 1]
                    path_bandwidths.append(bandwidth)
                    # compute end-to-end bandwidth rate
                    end_to_end_bandwidth = min(path_bandwidths)
                    path_latency += node_to_node_latency
                    # for each node further from original node, increase latency
                    if end_to_end_bandwidth == 0:
                        absolute_path_cost = 0
                    else:
                        # calculate absolute cost
                        absolute_path_cost = round(absolute_cost_function(bandwidth_data[current_node - 1][node - 1], path_latency), 3)
                    probability = ((trail_data[current_node][node - 1] * alpha) + \
                    ((1 - alpha) * absolute_path_cost)) \
                    / sum_of_probabilities
                    # do a cummulative probability that will be used to determine next path based
                    path_probabilities.append((node, path_probabilities[-1][1] + probability))
            rand_selector = random.uniform(0, 1)
            number_of_probs = len(path_probabilities)
            for idx in range(len(path_probabilities) - 1):
                if rand_selector > path_probabilities[idx][1] and rand_selector < path_probabilities[idx + 1][1]:
                    current_node = path_probabilities[idx + 1][0]
                    ant_data[ant]['paths'][-1]['path'].append(current_node)
                    break
                if idx + 1 == number_of_probs:
                    # if rand is number extremely close to one, select the last index
                    current_node = path_probabilities[idx + 1][0]
                    ant_data[ant]['paths'][-1]['path'].append(current_node)
                 
    return 

In [779]:
#traverse_nodes(ant_data, 15, x_y_data, bandwidth_data, trail_data)
#print(ant_data)

In [780]:
def remove_path_loops(ant_data):
    for ant, data in ant_data.items():
        latest_path = data['paths'][-1]['path']
        modified = True

        while modified:
            modified = False
            for idx in range(len(latest_path)):
                last_idx = len(latest_path) - 1
                while idx < last_idx:
                    if latest_path[idx] == latest_path[last_idx]:
                        latest_path = latest_path[:idx] + latest_path[last_idx:]
                        modified = True
                        break
                    last_idx -= 1
                if modified:
                    break 
        ant_data[ant]['paths'][-1]['path'] = latest_path

    return

In [781]:
#remove_path_loops(ant_data)

In [782]:
#ant_data

In [783]:
def calculate_path_total_cost(ant_data, bandwidth_data):
    for ant, data in ant_data.items():
        latest_path = data['paths'][-1]['path']
        path_cost = data['paths'][-1]['cost']
        path_latency = 0
        if len(latest_path) > 1:
            path_bandwidths = []
            for path in range(len(latest_path) - 1):
                path_bandwidths.append(bandwidth_data[latest_path[path] - 1][latest_path[path + 1] - 1])
                path_latency += node_to_node_latency
            # compute end-to-end bandwidth rate
            end_to_end_bandwidth = min(path_bandwidths)
            if end_to_end_bandwidth == 0:
                absolute_path_cost = 0
            else:
                # calculate absolute cost
                absolute_path_cost = round(absolute_cost_function(bandwidth_data[latest_path[path] - 1][latest_path[path + 1] - 1], path_latency), 3)
            ant_data[ant]['paths'][-1]['cost'] = absolute_path_cost  
            ant_data[ant]['paths'][-1]['end-to-end_transmission'] = end_to_end_bandwidth
            ant_data[ant]['paths'][-1]['latency'] = path_latency

    return

In [784]:
#calculate_path_total_cost(ant_data, bandwidth_data)

In [785]:
#ant_data

In [786]:
def negative_feedback(evaporation_rate, trail_data):
    for node, pheromone_data in trail_data.items():
        for idx, pheromone in enumerate(pheromone_data):
            trail_data[node][idx] = (1 - evaporation_rate) * pheromone

    return

In [787]:
#print(trail_data)

In [788]:
#negative_feedback(0.45, trail_data)

In [789]:
#print(trail_data)

In [790]:
def positive_feedback(Q, ant_data, trail_data, bandwidth_data):
    for ant, data in ant_data.items():
        latest_path = data['paths'][-1]['path']

        if len(latest_path) > 1:
            for idx in range(len(latest_path) - 1):
                # increase pheromone for path in both directions [a -> b] == [b -> a]
                if bandwidth_data[latest_path[idx] - 1][latest_path[idx + 1] - 1] == 0:
                    trail_data[latest_path[idx]][latest_path[idx + 1] - 1] += 0
                    trail_data[latest_path[idx + 1]][latest_path[idx] - 1] += 0
                else:
                    trail_data[latest_path[idx]][latest_path[idx + 1] - 1] += Q / bandwidth_data[latest_path[idx] - 1][latest_path[idx + 1] - 1]
                    trail_data[latest_path[idx + 1]][latest_path[idx] - 1] += Q / bandwidth_data[latest_path[idx] - 1][latest_path[idx + 1] - 1]

    return

In [791]:
#positive_feedback(1, ant_data, trail_data, bandwidth_data)

In [792]:
#print(trail_data)

In [793]:
def get_aco_data(origin, ant_data, bandwidth_data, aco_data, solutions):
    for ant, data in ant_data.items():
        if data['start_node'] == origin:
            for path in data['paths']:
                path_end_to_end_transmission = path['end-to-end_transmission']
                path_latency = path['latency']
                path_absolute_cost = path['cost']
                if path_absolute_cost > 0:
                    if len(aco_data['best_path']) == 0:
                        solutions[0] = {"source node": f"Node-{path['path'][0]}"}
                        routing_path = ''
                        for idx, sensor in enumerate(path['path'][1:]):
                            routing_path += f"(Node-{sensor}, {bandwidth_data[path['path'][idx] - 1][path['path'][idx + 1] - 1]} Mbps),"
                        solutions[0]["routing path"] = routing_path
                        solutions[0]["end-to-end transmission rate"] = f"{path_end_to_end_transmission} Mbps"
                        solutions[0]["end-to-end latency"] = f"{path_latency} ms"
                        solutions[0]["absolute path cost"] = f"{path_absolute_cost}"
                        aco_data['best_path'][str(path['path'])] = {'end_to_end_bandwidth': path_end_to_end_transmission,
                                                                         'path_latency': path_latency,
                                                                         'absolute_cost': path_absolute_cost
                                                                         }
                    else:
                        best_path = aco_data['best_path'][list(aco_data['best_path'].keys())[0]]
                        if path_absolute_cost > best_path['absolute_cost']:
                            solutions[0] = {"source node": f"Node-{path['path'][0]}"}
                            routing_path = ''
                            for idx, sensor in enumerate(path['path'][1:-1]):
                                routing_path += f"(Node-{sensor}, {bandwidth_data[idx][idx + 1]} Mbps),"
                            solutions[0]["routing path"] = routing_path
                            solutions[0]["end-to-end transmission rate"] = f"{path_end_to_end_transmission} Mbps"
                            solutions[0]["end-to-end latency"] = f"{path_latency} ms"
                            solutions[0]["absolute path cost"] = f"{path_absolute_cost}"
                            aco_data['possible_paths'][list(aco_data['best_path'].keys())[0]] = {
                                                     'end_to_end_bandwidth': best_path['end_to_end_bandwidth'],
                                                     'path_latency': best_path['path_latency'],
                                                     'absolute_cost': best_path['absolute_cost']
                                                    }
                            aco_data['best_path'].pop(list(aco_data['best_path'].keys())[0])
                            aco_data['best_path'][str(path['path'])] = {
                                                     'end_to_end_bandwidth': path_end_to_end_transmission,
                                                     'path_latency': path_latency,
                                                     'absolute_cost': path_absolute_cost
                                                    }
                        else:
                            aco_data['possible_paths'][str(path['path'])] = {'end_to_end_bandwidth': path_end_to_end_transmission,
                                                                         'path_latency': path_latency,
                                                                         'absolute_cost': path_absolute_cost
                                                                         }
    return aco_data, solutions

In [794]:
def ant_colony_optimization(origin, Q=1, evaporate=0.45, ants_count=20, max_iterations=10):
    trail_data = {}
    ant_data = {}
    aco_data = {'best_path': {},
            'possible_paths': {}}
    solutions = [{}]
    init_ants(origin, x_y_data, ants_count, ant_data)
    init_pheromone(x_y_data, trail_data)
    bandwidth_data = calculate_route_costs(x_y_data)
    for i in range(max_iterations):
        traverse_nodes(ant_data, x_y_data, bandwidth_data, trail_data)
        remove_path_loops(ant_data)
        calculate_path_total_cost(ant_data, bandwidth_data)
        negative_feedback(evaporate, trail_data)
        positive_feedback(Q, ant_data, trail_data, bandwidth_data)
    for ant, data in ant_data.items():
        print("ant {}: ({})".format(ant, data))
        print("--------------------------------------------------------")
    aco_data, solutions = get_aco_data(origin, ant_data, bandwidth_data, aco_data, solutions)
    return aco_data, solutions

In [795]:
#ant_colony_optimization(1)

ant 1: ({'start_node': 123, 'destination_node': 151, 'paths': [{'path': [123, 36, 127, 60, 100, 71, 67, 35, 126, 17, 18, 86, 38, 148, 115, 53, 116, 66, 15, 151], 'end-to-end_transmission': 0, 'latency': 570, 'cost': 0}, {'path': [123, 25, 86, 114, 98, 42, 28, 148, 121, 84, 150, 151], 'end-to-end_transmission': 0, 'latency': 330, 'cost': 0}, {'path': [123, 42, 8, 26, 93, 21, 43, 151], 'end-to-end_transmission': 0, 'latency': 210, 'cost': 0}, {'path': [123, 99, 125, 144, 102, 24, 54, 52, 127, 20, 8, 32, 107, 112, 60, 69, 109, 50, 114, 89, 100, 149, 18, 151], 'end-to-end_transmission': 0, 'latency': 690, 'cost': 0}, {'path': [123, 69, 8, 71, 95, 137, 7, 119, 151], 'end-to-end_transmission': 0, 'latency': 240, 'cost': 0}, {'path': [123, 80, 85, 139, 66, 45, 78, 122, 111, 117, 106, 107, 138, 73, 12, 61, 151], 'end-to-end_transmission': 0, 'latency': 480, 'cost': 0}, {'path': [123, 111, 44, 117, 69, 56, 63, 151], 'end-to-end_transmission': 0, 'latency': 210, 'cost': 0}, {'path': [123, 126, 3

({'best_path': {'[1, 100, 136, 87, 19, 62, 74, 61, 151]': {'end_to_end_bandwidth': 1,
    'path_latency': 240,
    'absolute_cost': 1.625}},
  'possible_paths': {'[1, 76, 150, 20, 86, 93, 94, 136, 92, 116, 138, 129, 43, 84, 28, 62, 61, 151]': {'end_to_end_bandwidth': 1,
    'path_latency': 510,
    'absolute_cost': 1.294},
   '[1, 100, 42, 94, 133, 55, 21, 106, 135, 19, 115, 79, 80, 141, 3, 152]': {'end_to_end_bandwidth': 1,
    'path_latency': 450,
    'absolute_cost': 1.333}}},
 [{'source node': 'Node-1',
   'routing path': '(Node-100, 0 Mbps),(Node-136, 0 Mbps),(Node-87, 0 Mbps),(Node-19, 0 Mbps),(Node-62, 0 Mbps),(Node-74, 0 Mbps),(Node-61, 0 Mbps),',
   'end-to-end transmission rate': '1 Mbps',
   'end-to-end latency': '240 ms',
   'absolute path cost': '1.625'}])